In [ ]:
# imports
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10

In [ ]:
#loading data and preprocessing

# Define the transformation for preprocessing the data

# define transforms for images
transform = transforms.Compose(
              [transforms.ToTensor(),
              transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Load the CIFAR-100 training and test datasets

cifar10_train = CIFAR10(root='./data', train=True, download=True, transform=transform)
cifar10_test = CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(cifar10_train, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(cifar10_test, batch_size=32, shuffle=True)

100%|██████████| 170498071/170498071 [00:06<00:00, 24873310.70it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
# define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import torchvision.models as models
# Load the EfficientNetB0 model
effnet = models.efficientnet_b0(pretrained=True)

# Remove the last layer (classifier)
effnet.classifier = torch.nn.Identity()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-3dd342df.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-3dd342df.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 60.3MB/s]


In [ ]:
class Cifar10ClassfierModel(nn.Module):
    def __init__(self, effnet, num_classes):
        super(Cifar10ClassfierModel, self).__init__()
        self.effnet = effnet
        self.relu = nn.ReLU()
        self.batch_norm = nn.BatchNorm2d(1280)
        self.dropout = nn.Dropout(p=0.5)
        self.dense1 = nn.Linear(1280, 640)
        self.dense2 = nn.Linear(640, 320)
        self.dense3 = nn.Linear(320, 160)
        self.dense4 = nn.Linear(160, 10)
        self.dense = nn.Linear(1280, num_classes)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.effnet(x)
        x = self.dense1(x)
        x = self.relu(x)
        x = self.dense2(x)
        x = self.relu(x)
        x = self.dense3(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.dense4(x)
        # x = self.sigmoid(x)
        return x

In [ ]:
import os

# Get the current working directory
current_path = os.getcwd()
# Load the trained weights
path=current_path+'/cifar10_classifier_model.pth'

pre_trained_classfier  = torch.load(path)


In [ ]:
#Testing  pre-trained model
correct = 0
total = 0
pre_trained_classfier.to(device)
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = pre_trained_classfier(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print('Accuracy on the test set: %.2f %%' % accuracy)

Accuracy on the test set: 84.94 %


In [ ]:
# define gausian noise
class GaussianNoiseLayer(nn.Module):
    def __init__(self, std_dev):
        super(GaussianNoiseLayer, self).__init__()
        self.std_dev = std_dev
    def forward(self, x):
        noise = torch.randn_like(x) * self.std_dev
        return x + noise

# define communication model
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.gaussian_noise = GaussianNoiseLayer(std_dev=0.0)
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 8, kernel_size=3, stride=2, padding=1),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(8, 16, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 3, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.Tanh()
        )


    def forward(self, x):
        x = self.encoder(x)
        x = (8*8*8)*x/torch.norm(x)
        x = self.gaussian_noise(x)
        x = self.decoder(x)

        return x


In [ ]:
import os

# Get the current working directory
current_path = os.getcwd()

# Load the trained weights
path=current_path+'/comm_model_std00.pt'

comm_model0  = torch.load(path)

comm_model0.to(device)

# Load the trained weights
path=current_path+'/comm_model_std01.pt'

comm_model1  = torch.load(path)

comm_model1.to(device)

# Load the trained weights
path=current_path+'/comm_model_std04.pt'

comm_model4  = torch.load(path)

comm_model4.to(device)

# Load the trained weights
path=current_path+'/comm_model_std06.pt'

comm_model6  = torch.load(path)

comm_model6.to(device)

Autoencoder(
  (gaussian_noise): GaussianNoiseLayer()
  (encoder): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(16, 8, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (3): ReLU()
  )
  (decoder): Sequential(
    (0): ConvTranspose2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
    (1): ReLU()
    (2): ConvTranspose2d(16, 3, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
    (3): Tanh()
  )
)

In [ ]:
#Testing pre-trained classfier on communication channel with gausian noise with std 0
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        # pre_trained_model_output=pre_trained_model(images)
        noise_image=comm_model0(images)
        outputs = pre_trained_classfier(noise_image)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print('Accuracy on the test set: %.2f %%' % accuracy)

Accuracy on the test set: 69.56 %


In [ ]:
#Testing pre-trained classfier on communication channel with gausian noise with std 0.1
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        # pre_trained_model_output=pre_trained_model(images)
        noise_image=comm_model1(images)
        outputs = pre_trained_classfier(noise_image)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print('Accuracy on the test set: %.2f %%' % accuracy)

Accuracy on the test set: 73.46 %


In [ ]:
#Testing pre-trained classfier on communication channel with gausian noise with std 0.4
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        # pre_trained_model_output=pre_trained_model(images)
        noise_image=comm_model4(images)
        outputs = pre_trained_classfier(noise_image)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print('Accuracy on the test set: %.2f %%' % accuracy)

Accuracy on the test set: 68.34 %


In [ ]:
#Testing pre-trained classfier on communication channel with gausian noise with std 0.6
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        # pre_trained_model_output=pre_trained_model(images)
        noise_image=comm_model6(images)
        outputs = pre_trained_classfier(noise_image)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print('Accuracy on the test set: %.2f %%' % accuracy)

Accuracy on the test set: 59.86 %
